__author__ =  'Julia Schmid'

# Das ist ein Template

In [17]:
# Installierung der erforderlichen Pakete
#pip install ...

## Einstellungen

In [18]:
# Importe
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler

In [19]:
# Alle Spalten eines Datensatzes anzeigen
pd.set_option('display.max_columns', None) 

## Daten einlesen

In [20]:
df = pd.read_csv("OnlineRetail.csv", encoding='ISO-8859-1')

## Daten verstehen

In [21]:
# Ausgabe der ersten 5 Zeilen
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [22]:
# Ausgabe der Anzahl der Zeilen und Spalten
print(f'Anzahl Zeilen: {df.shape[0]}')
print(f'Anzahl Spalten: {df.shape[1]}')

Anzahl Zeilen: 541909
Anzahl Spalten: 8


In [23]:
# Ausgabe der Datensatz-Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [24]:
# Ausgabe der Statistischenkennzahlen der numerischen Variablen
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [25]:
# Bestimmung der Numerische Variablen 
numerical_var = [col for col in df if df[col].dtype != 'object']
print(numerical_var)

# Bestimmung der Kategorische Variablen
categorical_var = [col for col in df if df[col].dtype == 'object']
print(categorical_var)

['Quantity', 'UnitPrice', 'CustomerID']
['InvoiceNo', 'StockCode', 'Description', 'InvoiceDate', 'Country']


In [26]:
# DATENQUALITÄT

## Daten aufbereiten

In [27]:
# Ausgabe des Namens der Variablen mit NaN-Werte und die Anzahl der Einträge mit NaN-Werten
count_nan = df.isna().sum()
count_nan[count_nan > 0]

Description      1454
CustomerID     135080
dtype: int64

In [28]:
df['Description'] = df['Description'].fillna('')

In [29]:
nextID = df['CustomerID'].max()

temp = {}

# Fehlende CustomerID auffüllen
for idx, row in df.iterrows():
    if pd.isna(row['CustomerID']):
        invoice_no = row['InvoiceNo']
        if invoice_no not in temp:
            temp[invoice_no] = nextID
            nextID += 1
        df.at[idx, 'CustomerID'] = temp[invoice_no]

In [30]:
df['Date'] = pd.to_datetime(df['InvoiceDate']).dt.date
df['Time'] = pd.to_datetime(df['InvoiceDate']).dt.time
df = df.drop(columns=['InvoiceDate'])

In [32]:
df['TotalPrice'] = df['UnitPrice']*df['Quantity']

In [33]:
df

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Time,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,08:26:00,15.30
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,08:26:00,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,08:26:00,20.34
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,0.85,12680.0,France,2011-12-09,12:50:00,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2.10,12680.0,France,2011-12-09,12:50:00,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,4.15,12680.0,France,2011-12-09,12:50:00,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,12680.0,France,2011-12-09,12:50:00,16.60
